In [136]:
import copy;

MISIONEROS = 3
CANIBALES = 3

ESTADO_INICIAL = [[0,0,False], [3,3,True]]
ESTADO_FINAL = [[3,3,True], [0,0,False]]

Un estado se compone de dos arrays, uno para cada orilla del río. Cada array cuenta con 2 enteros y un booleano que representan respectivamente el **número de misioneros**, el **número de caníbales** y la **posición de la barca** (True si está presente en esa orilla, False si está en la otra).

Cada una de las tuplas del array de 'movimientos' representan el número de misioneros y caníbales que se van a transportar en la barca.

In [137]:
def seComen(estado) -> bool:
  for orilla in estado:
    misioneros, canibales = orilla[0], orilla[1]
    if misioneros < canibales and misioneros >= 1 and canibales >= 1: return True
  return False

def transportar(estado, pasajeros: (int)):
  nMisioneros, nCanibales = pasajeros[0], pasajeros[1]
  e = copy.deepcopy(estado)
  if e[0][2]: #orilla IZQUIERDA
    for n in range(2):
      e[n][0] = e[n][0]-nMisioneros if n==0 else e[n][0]+nMisioneros
      e[n][1] = e[n][1]-nCanibales if n==0 else e[n][1]+nCanibales
      e[n][2] = True if n == 1 else False

  else:
    for n in range(1, -1, -1): #orilla DERECHA
      e[n][0] = e[n][0]-nMisioneros if n==1 else e[n][0]+nMisioneros
      e[n][1] = e[n][1]-nCanibales  if n==1 else e[n][1]+nCanibales
      e[n][2] = True if n==0 else False
  return e

def movimientoPosible(estado, movimiento: (int)) -> bool:
  if estado[0][2]: return estado[0][0] >= movimiento[0] and estado[0][1] >= movimiento[1]
  else: return estado[1][0] >= movimiento[0] and estado[1][1] >= movimiento[1]

def movimientos(estado):
  movimientos = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]
  estados = []
  for movimiento in movimientos:
    if movimientoPosible(estado, movimiento):
      resultado = transportar(estado, movimiento)
      if not seComen(resultado): estados.append(resultado)
  return estados

In [138]:
def pintarEstados(path):
  for e in path:
    derecha, izquierda = '', ''
    for n in range(2):
      if n == 0:
        m, c = e[n][0], e[n][1]
        izquierda = ' '.join(['m' for _ in range(m)] + ['c' for _ in range(c)])
      if n == 1:
        m, c = e[n][0], e[n][1]
        derecha = ' '.join(['m' for _ in range(m)] + ['c' for _ in range(c)])
    print("{:>15} | {:^5} | {:<15}".format(izquierda, '', derecha))

## Búsqueda en profundidad, **iterativa**.


In [139]:
path = []

abiertos, cerrados = [ESTADO_INICIAL], []
actual = abiertos[0]

while actual != ESTADO_FINAL and abiertos:
  path.append(actual)
  abiertos.remove(actual)
  cerrados.append(actual)
  hijos = movimientos(actual)
  hijos = [h for h in hijos if h not in cerrados]
  #abiertos = abiertos+[h for h in hijos if h not in abiertos] -> en anchura: (más lenta)
  abiertos = [h for h in hijos if h not in abiertos]+abiertos
  if abiertos:
    actual = abiertos[0]
  else:
    break

if  actual == ESTADO_FINAL:
  path.append(ESTADO_FINAL)
  for p in path:
    print(p)

[[0, 0, False], [3, 3, True]]
[[0, 2, True], [3, 1, False]]
[[0, 1, False], [3, 2, True]]
[[0, 3, True], [3, 0, False]]
[[0, 2, False], [3, 1, True]]
[[2, 2, True], [1, 1, False]]
[[1, 1, False], [2, 2, True]]
[[3, 1, True], [0, 2, False]]
[[3, 0, False], [0, 3, True]]
[[3, 2, True], [0, 1, False]]
[[2, 2, False], [1, 1, True]]
[[3, 3, True], [0, 0, False]]


In [140]:
pintarEstados(path)

                |       | m m m c c c    
            c c |       | m m m c        
              c |       | m m m c c      
          c c c |       | m m m          
            c c |       | m m m c        
        m m c c |       | m c            
            m c |       | m m c c        
        m m m c |       | c c            
          m m m |       | c c c          
      m m m c c |       | c              
        m m c c |       | m c            
    m m m c c c |       |                


## **Dijkstra**.


In [141]:
def tupla(estado):
  return tuple(map(tuple, estado))

def Dijkstra(ESTADO_INICIAL, ESTADO_FINAL):
  D = {}
  abiertos, cerrados = [ESTADO_INICIAL], []
  D[tupla(ESTADO_INICIAL)] = {"pasos": 0, "padre": None}

  while abiertos:
    m = min(abiertos, key=lambda x: D[tupla(x)]['pasos'])
    actual = abiertos.pop(abiertos.index(m))
    cerrados.append(actual)
    a = tupla(actual)
    if a not in D:
        D[a] = {"pasos": float("+inf"), "padre": None}
    if actual == ESTADO_FINAL:
        path=[]
        actual=tupla(ESTADO_FINAL)

        while actual!=None:
            path.append(actual)
            actual=D[actual]["padre"]
        return path[::-1]

    for hijo in movimientos(actual):
        h = tupla(hijo)
        if h not in D:
            d = D[a]['pasos'] + 1
            D[h] = {"pasos": d, "padre": a}
        if not hijo in cerrados:
            if D[h]['pasos'] >= D[a]['pasos'] + 1:
                D[h]['pasos'] = D[a]['pasos'] + 1
                abiertos.append(hijo)
  return None

path = Dijkstra(ESTADO_INICIAL, ESTADO_FINAL)
if path is not None:
  for p in path:
    print(p)
else: print('Final no encontrado')

((0, 0, False), (3, 3, True))
((0, 2, True), (3, 1, False))
((0, 1, False), (3, 2, True))
((0, 3, True), (3, 0, False))
((0, 2, False), (3, 1, True))
((2, 2, True), (1, 1, False))
((1, 1, False), (2, 2, True))
((3, 1, True), (0, 2, False))
((3, 0, False), (0, 3, True))
((3, 2, True), (0, 1, False))
((2, 2, False), (1, 1, True))
((3, 3, True), (0, 0, False))


In [142]:
pintarEstados(path)

                |       | m m m c c c    
            c c |       | m m m c        
              c |       | m m m c c      
          c c c |       | m m m          
            c c |       | m m m c        
        m m c c |       | m c            
            m c |       | m m c c        
        m m m c |       | c c            
          m m m |       | c c c          
      m m m c c |       | c              
        m m c c |       | m c            
    m m m c c c |       |                
